In [1]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2() 

Conexion exitosa con la Base de datos:irvin_hisminsa


In [34]:
#consulta a la base de datos altitud de establecimiento 
def consulta_paquete(conn,grupo):
    paquete=conn.df("""SELECT * FROM indicadores.reglas_validacion rv where etapas_vida=%s """ % grupo)   
    return paquete

def consulta_paquete_detalle(conn,id):
    paquete=conn.df("""SELECT * FROM indicadores.reglas_validacion_detalles rvd WHERE rvd.id_reglas_validadcion =%s"""%id)   
    return paquete 

In [ ]:

def consulta_hisminsa(conn):  
    sql = conn.df(""" SELECT * FROM maestros.nominal_trama nt WHERE nt.fecha_atencion  BETWEEN '2023-07-01' AND '2023-08-20' AND id_otra_condicion =1  """)
    return sql
df_hisminsa=consulta_hisminsa(conn)

In [ ]:
df=df_hisminsa
# Eliminar duplicados
df_no_duplicates = df.drop_duplicates(subset=['id_paciente', 'codigo_item','fecha_atencion','id_correlativo_lab'])

# Pivoteo dinámico usando pandas
pivoted_df = df_no_duplicates.pivot(index=['id_cita','id_paciente','fecha_atencion','codigo_item', 'tipo_diagnostico'],
                                     columns='id_correlativo_lab',
                                     values='valor_lab').reset_index()
print(pivoted_df)
#pivoted_df.to_excel('demo.xlsx', index=False)

<h1>Materno</h1>

In [35]:
data = {'id_paciente': []}
df = pd.DataFrame(data)
df_Paquete = consulta_paquete(conn,1)

for index, row1 in df_Paquete.iterrows():
    id_p = row1['id']
    df_DPaquete=consulta_paquete_detalle(conn,id_p)
    for index, row in df_DPaquete.iterrows():
        dx = row['codigo_item']
        tdx = row['tipo_diagnostico']
        desc = row['descripcion']
        lab1= "AND pd.lab1='"+row['lab_1']+"'" if row['lab_1'] is not None  else ''
        lab2="AND pd.lab2='"+row['lab_2']+"'"  if row['lab_2'] is not None  else ''
        lab3="AND pd.lab3='"+row['lab_3']+"'"  if row['lab_3'] is not None  else ''   

        sql = """SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '%s' %s %s %s AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.id_otra_condicion=1""" % (dx, lab1, lab2, lab3)
        print(sql)
        sql = conn.df(sql)
        sql['fecha_atencion'] = pd.to_datetime(sql['fecha_atencion'], format='%Y-%m-%d')
        sql['fecha_atencion'] = sql['fecha_atencion'].dt.date
        sql['id_paciente'] = pd.to_numeric(sql['id_paciente'], errors='coerce')
        # INDEXAR
        sql = sql.set_index('id_paciente')
            # extraer el valosr maximo
        sql = sql.groupby(sql.index).tail(1)
        
        lab1= "|"+row['lab_1'] if row['lab_1'] is not None  else ''
       # dx2 = dx.replace('.', '_')
        dx2 = dx
        dx2 = """%s.- %s \n %s%s"""%(id_p,desc,dx2,lab1)
        sql.rename(columns={'fecha_atencion': dx2}, inplace=True)
        
        df = pd.merge(df, sql, on='id_paciente', how='outer')
        
        #print(df)
df = df.set_index('id_paciente')
df=df.reset_index()


    



    

d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '99199.17' AND pd.lab1='SF1'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.id_otra_condicion=1
SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '99199.17' AND pd.lab1='SF2'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.id_otra_condicion=1


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '99199.17' AND pd.lab1='SF3'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.id_otra_condicion=1


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


KeyboardInterrupt: 

In [36]:
data = {'id_paciente': []}
df = pd.DataFrame(data)
df_Paquete = consulta_paquete(conn,2)

for index, row1 in df_Paquete.iterrows():
    id_p = row1['id']
    df_DPaquete=consulta_paquete_detalle(conn,id_p)




    for index, row in df_DPaquete.iterrows():
        dx = row['codigo_item']
        tdx = row['tipo_diagnostico']
        desc = row['descripcion']
        lab1= "AND pd.lab1='"+row['lab_1']+"'" if row['lab_1'] is not None  else ''
        lab2="AND pd.lab2='"+row['lab_2']+"'"  if row['lab_2'] is not None  else ''
        lab3="AND pd.lab3='"+row['lab_3']+"'"  if row['lab_3'] is not None  else ''   

        sql = """SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '%s' %s %s %s AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1""" % (dx, lab1, lab2, lab3)
        print(sql)
        sql = conn.df(sql)
        sql['fecha_atencion'] = pd.to_datetime(sql['fecha_atencion'], format='%Y-%m-%d')
        sql['fecha_atencion'] = sql['fecha_atencion'].dt.date
        sql['id_paciente'] = pd.to_numeric(sql['id_paciente'], errors='coerce')
        # INDEXAR
        sql = sql.set_index('id_paciente')
            # extraer el valosr maximo
        sql = sql.groupby(sql.index).tail(1)
        
        lab1= "|"+row['lab_1'] if row['lab_1'] is not None  else ''
       # dx2 = dx.replace('.', '_')
        dx2 = dx
        dx2 = """%s.- %s \n %s%s"""%(id_p,desc,dx2,lab1)
        sql.rename(columns={'fecha_atencion': dx2}, inplace=True)
        
        df = pd.merge(df, sql, on='id_paciente', how='outer')
        
        #print(df)
df = df.set_index('id_paciente')
df=df.reset_index()


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '99199.17' AND pd.lab1='SF1'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1
SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '99199.17' AND pd.lab1='SF2'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '99199.17' AND pd.lab1='SF3'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1
SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '99199.17' AND pd.lab1='TA'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../.

...

SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '90749.03' AND pd.lab1='1'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '90657' AND pd.lab1='2'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1
SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '90749.03' AND pd.lab1='2'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../.

SELECT pd.id_paciente, pd.fecha_atencion FROM indicadores.paciente_diagnostico pd WHERE pd.codigo_item = '85018' AND pd.lab1='1'   AND pd.fecha_atencion BETWEEN '2023-07-01' AND '2023-07-31' and pd.anio_actual_paciente<1


In [37]:
def consulta_pacientes(conn):
    paquete=conn.df(""" SELECT mp.id_paciente ,mhtd.abrev_tipo_doc,mp.numero_documento,mp.genero, mp.fecha_nacimiento,mp.id_pais   FROM maestros.maestro_paciente mp 
INNER JOIN maestros.maestro_his_tipo_doc mhtd ON mp.id_tipo_documento =mhtd.id_tipo_documento  """)   
    paquete['id_paciente'] = paquete['id_paciente'].astype('int64')
    return paquete
paciente_df =consulta_pacientes(conn)


d:\Irvin\Irvin\Python\pandas\modulos\pruebas\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [38]:
df_2=paciente_df.merge(df, how='inner', on=['id_paciente'])
df_2.head()

,id_paciente,abrev_tipo_doc,numero_documento,genero,fecha_nacimiento,id_pais,48.- SULFATO FERROSO \n 99199.17|SF1,48.- SULFATO FERROSO \n 99199.17|SF2,48.- SULFATO FERROSO \n 99199.17|SF3,48.- SULFATO FERROSO \n 99199.17|TA,...,27.- Rotavirus \n 90681|1,27.- Antipoliomielitica (IPV) \n 90713|2,27.- DPT-HvB-HiB \n 90723|2,27.- Antineumocócica \n 90670|2,27.- Rotavirus \n 90681|2,27.- influenza \n 90657|1,27.- COVID \n 90749.03|1,27.- influenza \n 90657|2,27.- COVID \n 90749.03|2,47.- Dosaje de HB a prematuros \n 85018|1
0,511011832299,DNI,93142589,M,2022-11-16,PER,NaN,NaN,2023-07-18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,511019692388,DNI,93136161,F,2022-11-11,PER,NaN,2023-07-13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-13,NaN,NaN
2,511052912535,DNI,93140626,F,2022-11-15,PER,NaN,2023-07-14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,511056952288,DNI,93129952,M,2022-11-08,PER,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,511084862353,DNI,93148160,M,2022-11-21,PER,NaN,NaN,2023-07-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
pd.options.display.float_format = '{:,.0f}'.format

df_2.to_excel('Ninio.xlsx', index=False)


In [28]:
print(df.dtypes)

id_paciente                                       int64
1.- HVB \n 90746                                 object
1.- H1N1 \n 90658                                object
1.- DTPA \n 90715                                object
2.- GESTANTE CON REGISTRO  \n Z3491              object
                                                  ...  
4.- 2DO PLAN DE PARTO  \n 59401.06|2             object
4.- 3RO PLAN DE PARTO  \n 59401.06|3             object
21.- TOTAL DE GESTANTES CON REGISTRO \n C0011    object
21.- TAMIZAJE DE SÍFILIS \n 86318.01             object
21.- TAMIZAJE DE SÍFILIS \n 86780.01             object
Length: 96, dtype: object
